In [ ]:
import numpy as np 
import pandas as pd 

dataset = pd.read_csv("IT_2013p_EUSILC.csv")

dataset.head()

,PB010,PB020,PB030,PB040,PB060,PB100,PB110,PB120,PB130,PB140,...,PW190,PW200,PW210,PW220,PX010,PX020,PX030,PX040,PX050,PX200
0,2013,IT,821,1431.730769,NaN,2,2013,10.0,1,1932,...,8.0,8.0,8.0,NaN,1,80,82,1,6.0,543.652549
1,2013,IT,1031,1431.730769,NaN,3,2013,14.0,4,1964,...,6.0,9.0,8.0,NaN,1,48,103,1,2.0,1753.075882
2,2013,IT,1371,1431.730769,NaN,4,2013,16.0,2,1979,...,6.0,8.0,8.0,NaN,1,33,137,1,6.0,NaN
3,2013,IT,1631,1431.730769,NaN,4,2013,1.0,2,1953,...,6.0,NaN,NaN,2.0,1,59,163,1,2.0,8765.280972
4,2013,IT,1831,1431.730769,NaN,2,2013,9.0,3,1932,...,NaN,8.0,NaN,3.0,1,80,183,1,6.0,7884.221284
